# Aspect Based Sentiment Analysis

- This Notebook Contains out Bi-LSTM Based Aspect Classification Model + Other LSTM + Attention Based Models we experimented with.

#### References:
- https://github.com/mjain72/Sentiment-Analysis-using-Word2Vec-and-LSTM/blob/master/SentimentAnalysisTwitter.py
- https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention
- https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
- https://github.com/rohit-gupta/POS-Attention/blob/master/pos-attention-model.py

"""
AIT726 Final project - Part 2 -  Aspect Classification using Deep learning models on SemEval’16 dataset ( 1708 training dataset and 587 testing dataset ) and Foursquare ( 849 testing dataset )

Authors: Yasas, Prashanti, Ashwini

Command to run the file: run DL_MODEL_AC.ipynb

Note : For more details please check README file
"""

In [0]:
# Setup Notebook
%matplotlib inline

## Setup Requirements

- Load Libraries and Required Resources

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_validate, KFold, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.metrics import confusion_matrix, roc_curve,  roc_auc_score, classification_report, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention, Bidirectional
from tensorflow.keras.layers import Lambda, dot, Activation, concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import Callback
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Load From Project
from absa.config import DATA_PATHS
from absa.dataset import load_dataset

In [0]:
import spacy
#Pass sentences through spacy nlp pipeline and get the output terms
nlp = spacy.load('en')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#Create a set of stopwords
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 
create word2vec (  pre trained word embeddings ) from 
GoogleNews-vectors-negative300.bin.gz



In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/resources/GoogleNews-vectors-negative300.bin', binary=True)

'Word2Vec Vector Size: %d' % word2vec.vector_size

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


'Word2Vec Vector Size: 300'

    """Load and display semeval16 training dataset
       Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""
       

In [0]:
train_ds_path = DATA_PATHS['asba.semeval16.raw.train']

df_train = load_dataset(train_ds_path)

df_train = pd.DataFrame({
    'text': df_train.groupby('id')['text'].first(),
    'categories': df_train.groupby('id')['category'].apply(list),
})

df_train.head()

,text,categories
id,,
1004293:0,Judging from previous posts this used to be a ...,[RESTAURANT#GENERAL]
1004293:1,"We, there were four of us, arrived at noon - t...",[SERVICE#GENERAL]
1004293:2,"They never brought us complimentary noodles, i...",[SERVICE#GENERAL]
1004293:3,The food was lousy - too sweet or too salty an...,"[FOOD#QUALITY, FOOD#STYLE_OPTIONS]"
1004293:4,"After all that, they complained to me about th...",[SERVICE#GENERAL]


    """Load and display semeval16 testing dataset
       Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [0]:
test_ds_path = DATA_PATHS['asba.semeval16.raw.test.gold']

df_test = load_dataset(test_ds_path)

df_test = pd.DataFrame({
    'text': df_test.groupby('id')['text'].first(),
    'categories': df_test.groupby('id')['category'].apply(list),
})

df_test.head()

,text,categories
id,,
en_BlueRibbonSushi_478218171:0,Yum!,[FOOD#QUALITY]
en_BlueRibbonSushi_478218171:1,Serves really good sushi.,[FOOD#QUALITY]
en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,[FOOD#STYLE_OPTIONS]
en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,[FOOD#QUALITY]
en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,[FOOD#QUALITY]


    """Load and display Foursquare testing dataset
       Load - Load dataset using load_dataset method ( Reads formatted XML file from the provided path )"""

In [0]:
test_fs_ds_path = DATA_PATHS['asba.foursquare.raw.test.gold']

df_test_fs = load_dataset(test_fs_ds_path)

df_test_fs = pd.DataFrame({
    'text': df_test_fs.groupby('id')['text'].first(),
    'categories': df_test_fs.groupby('id')['category'].apply(list),
})

df_test_fs.head()

,text,categories
id,,
0:0,2 words -filter coffee :-),[DRINKS#QUALITY]
101:0,Absolute favourite hotpot in town!,[RESTAURANT#GENERAL]
102:0,"Great great food, not too pricey, good service...","[FOOD#QUALITY, SERVICE#GENERAL, LOCATION#GENER..."
102:1,Totally recommended.,[RESTAURANT#GENERAL]
103:0,Go to sandwich bar for lunch and split it with...,"[FOOD#STYLE_OPTIONS, FOOD#PRICES]"


## Preprocess

Preprocess Targets

    """As we have multiple aspects we have used MultiLabelBinarizer to create y_train, y_test and y_test_fs"""


In [0]:
mlb = MultiLabelBinarizer()

y_train = mlb.fit_transform(df_train.categories)

y_test = mlb.transform(df_test.categories)

y_test_fs = mlb.transform(df_test_fs.categories)

y_train.shape, y_test.shape, y_test_fs.shape

((1708, 12), (587, 12), (849, 12))

### Preprocess Features

""" Preprocess features - Train and test data , pos are passed through spacy_doc pipeline to generate the list of tokens, which are later passed through keras tokenizer to convert them to integers. Finally they are padded to maxlenght . By performing above steps we have our x_train, x_test and x_test_fs input features """

In [0]:
maxlen = 50

train_tokens = []
train_pos_tags = []

for sent in df_train.text:
  spacy_doc = nlp(sent)
  train_tokens.append([t.text for t in spacy_doc])
  train_pos_tags.append([t.pos_ for t in spacy_doc])

test_tokens = []
test_pos_tags = []

for sent in df_test.text:
  spacy_doc = nlp(sent)
  test_tokens.append([t.text for t in spacy_doc])
  test_pos_tags.append([t.pos_ for t in spacy_doc])

test_fs_tokens = []
test_fs_pos_tags = []

for sent in df_test_fs.text:
  spacy_doc = nlp(sent)
  test_fs_tokens.append([t.text for t in spacy_doc])
  test_fs_pos_tags.append([t.pos_ for t in spacy_doc])

#Convert words to integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tokens + test_tokens + test_fs_tokens)
x_train = tokenizer.texts_to_sequences(train_tokens)
x_test = tokenizer.texts_to_sequences(test_tokens)
x_test_fs = tokenizer.texts_to_sequences(test_fs_tokens)

#Convert POS to integers
pos_tokenizer = Tokenizer()
pos_tokenizer.fit_on_texts(train_pos_tags + test_pos_tags + test_fs_pos_tags)
x_pos_train = pos_tokenizer.texts_to_sequences(train_pos_tags)
x_pos_test = pos_tokenizer.texts_to_sequences(test_pos_tags)
x_pos_test_fs = pos_tokenizer.texts_to_sequences(test_fs_pos_tags)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_pos_train = pad_sequences(x_pos_train, maxlen=maxlen)

x_test = pad_sequences(x_test, maxlen=maxlen)
x_pos_test = pad_sequences(x_pos_test, maxlen=maxlen)

x_test_fs = pad_sequences(x_test_fs, maxlen=maxlen)
x_pos_test_fs = pad_sequences(x_pos_test_fs, maxlen=maxlen)

x_train.shape, x_test.shape, x_test_fs.shape

((1708, 50), (587, 50), (849, 50))

## Create Model

""" create word embeddings and embedding_index based on word2vec """

In [0]:
word_index = tokenizer.word_index

seq_length = maxlen
embedding_dim = word2vec.syn0.shape[1]
vocab_size = len(word_index) + 1

embeddings_index = word2vec

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
  if word in embeddings_index:
    embedding_vector = embeddings_index[word]
    embedding_matrix[i] = embedding_vector
  else:
      pass # for words not in embedding index

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [0]:
#Extract vocabulary size and word embedding dimension
max_tokens, dimension = len(pos_tokenizer.word_index) + 1, 300

""" Metrics on_train_begin and on_epoch_end are callback functions used to print intermediate results during training"""

In [0]:
class Metrics(Callback):
  def __init__(self, model, validation_data):
    self.model_ = model
    self.validation_data = validation_data

  def on_train_begin(self, logs={}):
    print('epoch, precision_micro, recall_micro, f1_micro')
  
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model_.predict(self.validation_data[0]))).round()
    val_targ = self.validation_data[1]
    _val_precision = precision_score(val_targ, val_predict, average='micro')
    _val_recall = recall_score(val_targ, val_predict, average='micro')
    _val_f1 = f1_score(val_targ, val_predict, average='micro')
    print('%d, %f, %f, %f' % (epoch + 1, _val_precision, _val_recall, _val_f1))

""" Attention layer is created , reference to the source code usage has been provided """

In [0]:
# from https://github.com/philipperemy/keras-attention-mechanism/blob/master/attention/attention.py
def attention_3d_block(hidden_states):
    """
    Many-to-one attention mechanism for Keras.
    @param hidden_states: 3D tensor with shape (batch_size, time_steps, input_dim).
    @return: 2D tensor with shape (batch_size, 128)
    @author: felixhao28.
    """
    hidden_size = int(hidden_states.shape[2])
    # Inside dense layer
    #              hidden_states            dot               W            =>           score_first_part
    # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
    # W is the trainable weight matrix of attention Luong's multiplicative style score
    score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
    #            score_first_part           dot        last_hidden_state     => attention_weights
    # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
    h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
    score = dot([score_first_part, h_t], [2, 1], name='attention_score')
    attention_weights = Activation('softmax', name='attention_weight')(score)
    # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
    context_vector = dot([hidden_states, attention_weights], [1, 1], name='context_vector')
    pre_activation = concatenate([context_vector, h_t], name='attention_output')
    # attention_vector = Dense(128, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
    return pre_activation

"""create_model - Create different models such as 'LSTM', 'self-attention', 'pos-attention', 'bidirectional. For all the models we have one layer of 128 hidden units, and a fully connected output layer using sigmoid as activation function. We have used Adam optimizer, and cross-entropy for the loss function with learning rate 0.001 for all the models."""

In [0]:
lstm_dropout = 0.8
lstm_out = 128
use_model =  ['LSTM', 'self-attention', 'pos-attention', 'bidirectional'][3]
loss = 'binary_crossentropy'
def create_model():
  embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=seq_length)

  optimizer = optimizers.Adam(learning_rate=0.001)

  # Word Embedding
  word_input_layer = Input(shape=(seq_length,))
  word_embedding_output = embedding_layer(word_input_layer)

  # POS Embedding
  if use_model == 'pos-attention':
    pos_embedding_layer = Embedding(max_tokens, dimension, input_length=seq_length)
    pos_input_layer = Input(shape=(seq_length,), name='pos_seq')
    pos_embedding_output = pos_embedding_layer(pos_input_layer)

  # word_pos_attention_seq = Attention()([word_embedding_output, pos_embedding_output])

  if use_model == 'self-attention':
    lstm_output = LSTM(units=lstm_out, dropout=lstm_dropout, return_sequences=True)(word_embedding_output)
    lstm_output = attention_3d_block(lstm_output)
  elif use_model == 'pos-attention':
    pass
  elif use_model == 'bidirectional':
    lstm_output = Bidirectional(LSTM(units=lstm_out, dropout=lstm_dropout))(word_embedding_output)
  else:
    lstm_output = LSTM(units=lstm_out, dropout=lstm_dropout)(word_embedding_output)

  output_layer = Dense(12, activation='sigmoid')(lstm_output)

  model = Model(inputs=[word_input_layer], outputs=[output_layer])
  model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
  return model

## Cross Validate

"""Perform 5-fold cross validation on the training dataset and use Metrics to derive intermediate evaluation results and finally we are using these results  we perform error analysis """

---



In [0]:
if False: # Done
    # Five fold Cross-Validation 
    y_preds = np.zeros(y_train.shape)
    kf = KFold(n_splits=5)
    for i, (train_index, valid_index) in enumerate(kf.split(x_train)):
        model, metrics = None, None
        print('Epoch: %d' % i)
        x_train_1, x_valid, y_train_1, y_valid = x_train[train_index], x_train[valid_index], y_train[train_index], y_train[valid_index]
        model = create_model()
        metrics = Metrics(model, validation_data=(x_valid, y_valid))
        model.fit(x_train_1, y_train_1, epochs=num_epochs, verbose=0, batch_size=batch_size, callbacks=[metrics])
        # predict the results
        score, acc = model.evaluate(x_valid, y_valid, verbose=2, batch_size=batch_size)
        y_pred = model.predict(x_valid)
        y_preds[valid_index] = y_pred
        y_pred_bool = y_pred > threshold
        # ROC AUC curve
        roc_auc = roc_auc_score(y_valid, y_pred)
        f1_micro = f1_score(y_valid, y_pred_bool, average='micro')
        print("roc_auc, {}".format(roc_auc))
        report = classification_report(y_valid, y_pred_bool, target_names=list(mlb.classes_), output_dict=True)
        df = pd.DataFrame(report).transpose()
        print(df.to_csv())
    df_train['predictions'] = [list(x) for x in mlb.inverse_transform(y_preds > 0.5)]
    df_train.to_excel('.project/output/preds/ac_dl.xlsx')

## Train Model

In [0]:
#fit model
batch_size = 32
num_epochs = 15

model = create_model()
print(model.summary())

x_train_1, x_valid, y_train_1, y_valid = train_test_split(x_train, y_train, test_size= 0.2, random_state = 24)
metrics = Metrics(model, validation_data=(x_valid, y_valid))
model.fit(x_train_1, y_train_1, epochs=num_epochs, batch_size=batch_size, validation_data=(x_valid, y_valid))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 300)           1341900   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               439296    
_________________________________________________________________
dense (Dense)                (None, 12)                3084      
Total params: 1,784,280
Trainable params: 1,784,280
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
43/43 [==============================] - 2s 42ms/step - loss: 0.3581 - accuracy: 0.2943 - val_loss: 0.2857 - val_accuracy: 0.2719
Epoch 2/15
43/43 [==============================] - 1s 21ms/step - loss: 0.2772 - accuracy: 0.3755 - va

""" Fit model on the complete training data set """"




In [0]:
#fit model
batch_size = 32
num_epochs = 15

model = create_model()
print(model.summary())

model.fit(x_train, y_train, epochs=num_epochs, verbose=0, batch_size=batch_size)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 300)           1341900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
Total params: 1,784,280
Trainable params: 1,784,280
Non-trainable params: 0
_________________________________________________________________
None




```
# This is formatted as code
```

## Dev Set Validate

In [0]:
threshold = 0.5

# predict the results
score, acc = model.evaluate(x_valid, y_valid, verbose=0, batch_size=batch_size)
y_pred = model.predict(x_valid)

y_pred_bool = y_pred > threshold

# ROC AUC curve

roc_auc = roc_auc_score(y_valid, y_pred)
f1_micro = f1_score(y_valid, y_pred_bool, average='micro')

print("roc_auc, {}".format(roc_auc))

report = classification_report(y_valid, y_pred_bool, target_names=list(mlb.classes_), output_dict=True)

df = pd.DataFrame(report).transpose()

print(df.to_csv())

roc_auc, 0.9697943753287932
,precision,recall,f1-score,support
AMBIENCE#GENERAL,0.9787234042553191,0.9583333333333334,0.968421052631579,48.0
DRINKS#PRICES,1.0,0.3333333333333333,0.5,3.0
DRINKS#QUALITY,1.0,0.2727272727272727,0.42857142857142855,11.0
DRINKS#STYLE_OPTIONS,0.0,0.0,0.0,10.0
FOOD#PRICES,0.9090909090909091,0.625,0.7407407407407406,16.0
FOOD#QUALITY,0.9558823529411765,0.9027777777777778,0.9285714285714286,144.0
FOOD#STYLE_OPTIONS,0.8,0.6,0.6857142857142857,20.0
LOCATION#GENERAL,0.0,0.0,0.0,2.0
RESTAURANT#GENERAL,0.922077922077922,0.9102564102564102,0.9161290322580644,78.0
RESTAURANT#MISCELLANEOUS,1.0,0.36363636363636365,0.5333333333333333,22.0
RESTAURANT#PRICES,0.7333333333333333,0.7333333333333333,0.7333333333333333,15.0
SERVICE#GENERAL,0.9875,0.9634146341463414,0.9753086419753086,82.0
micro avg,0.9440203562340967,0.8226164079822617,0.8791469194312796,451.0
macro avg,0.7738839934748883,0.555234371545347,0.6175102730941252,451.0
weighted avg,0.9203288574175548,0.82261640798226

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Evaluate on the test data 

- Do not use this for comparing your models

In [0]:
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 300)           1341900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
Total params: 1,784,280
Trainable params: 1,784,280
Non-trainable params: 0
_________________________________________________________________
None


### SemEval-16 Test Data

""" Model predict on SemEval-16 test dataset"""





In [0]:
threshold = 0.5

# predict the results
score, acc = model.evaluate(x_test, y_test, verbose=0, batch_size=batch_size)
y_pred = model.predict(x_test)
y_pred_bool = y_pred > threshold
# ROC AUC curve
roc_auc = roc_auc_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred_bool, average='micro')

print("roc_auc, {}".format(roc_auc))

report = classification_report(y_test, y_pred_bool, target_names=list(mlb.classes_), output_dict=True)

df = pd.DataFrame(report).transpose()

print(df.to_csv())

roc_auc, 0.9196456602764628
,precision,recall,f1-score,support
AMBIENCE#GENERAL,0.6470588235294118,0.7719298245614035,0.704,57.0
DRINKS#PRICES,0.0,0.0,0.0,3.0
DRINKS#QUALITY,1.0,0.047619047619047616,0.0909090909090909,21.0
DRINKS#STYLE_OPTIONS,0.0,0.0,0.0,12.0
FOOD#PRICES,0.6153846153846154,0.36363636363636365,0.4571428571428572,22.0
FOOD#QUALITY,0.8678414096916299,0.8716814159292036,0.869757174392936,226.0
FOOD#STYLE_OPTIONS,0.4482758620689655,0.2708333333333333,0.33766233766233766,48.0
LOCATION#GENERAL,0.0,0.0,0.0,13.0
RESTAURANT#GENERAL,0.824,0.7253521126760564,0.7715355805243446,142.0
RESTAURANT#MISCELLANEOUS,0.0,0.0,0.0,33.0
RESTAURANT#PRICES,0.6428571428571429,0.42857142857142855,0.5142857142857143,21.0
SERVICE#GENERAL,0.9044117647058824,0.8482758620689655,0.8754448398576512,145.0
micro avg,0.8084415584415584,0.6702557200538358,0.7328918322295805,743.0
macro avg,0.49581913485313733,0.36065828236631686,0.3850614662312443,743.0
weighted avg,0.7412085065028394,0.6702557200538358,0.6

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 Foursquare Test Data



""" Model predict on  test Foursqure dataset"""

In [0]:
threshold = 0.5
# predict the results
y_pred_fs = model.predict(x_test_fs)
y_pred_fs_bool = y_pred_fs > threshold
# ROC AUC curve
roc_auc = roc_auc_score(y_test_fs, y_pred_fs)
print("roc_auc, {}".format(roc_auc))
# Classification Report
report = classification_report(y_test_fs, y_pred_fs_bool, target_names=list(mlb.classes_), output_dict=True)
df = pd.DataFrame(report).transpose()
print(df.to_csv())

roc_auc, 0.8726199915650126
,precision,recall,f1-score,support
AMBIENCE#GENERAL,0.7090909090909091,0.582089552238806,0.6393442622950819,67.0
DRINKS#PRICES,1.0,0.16666666666666666,0.2857142857142857,6.0
DRINKS#QUALITY,1.0,0.01282051282051282,0.02531645569620253,78.0
DRINKS#STYLE_OPTIONS,0.0,0.0,0.0,11.0
FOOD#PRICES,0.2857142857142857,0.09523809523809523,0.14285714285714285,21.0
FOOD#QUALITY,0.8393285371702638,0.7954545454545454,0.8168028004667445,440.0
FOOD#STYLE_OPTIONS,0.43902439024390244,0.3333333333333333,0.37894736842105264,54.0
LOCATION#GENERAL,0.0,0.0,0.0,14.0
RESTAURANT#GENERAL,0.4581005586592179,0.5694444444444444,0.5077399380804953,144.0
RESTAURANT#MISCELLANEOUS,0.0,0.0,0.0,23.0
RESTAURANT#PRICES,0.5882352941176471,0.3125,0.40816326530612246,32.0
SERVICE#GENERAL,0.9029126213592233,0.6458333333333334,0.7530364372469636,144.0
micro avg,0.7250608272506083,0.5764023210831721,0.6422413793103449,1034.0
macro avg,0.5185338830296207,0.29278170696081146,0.3298268296736743,1034.0
weight

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Save the Model

""" Save the required model, tokenizer and mlb for demo purposes """


In [0]:
import pickle

model.save('/content/project/output/models/dl_model_ac.h5')

# saving tokenizer
with open('/content/project/output/models/tokenizer_ac.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# saving mlb
with open('/content/project/output/models/mlb_ac.pickle', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)